In [1]:
import torch

import mlflow

import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from baseline import BaselineModel

In [4]:
batch_size=4
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])
train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

validation_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [5]:
baseline_model = BaselineModel()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
baseline_model.to(device)

BaselineModel(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [6]:
n_epochs = 3
lr = 0.001
criterion=nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(baseline_model.parameters(), lr=lr)

In [8]:
loss_p = np.array([])
accuracy_p = np.array([])
for epoch in range(n_epochs):
    
    #training
    baseline_model.train()
    total_image = 0
    correct_image = 0
    running_loss = 0
    for i, data in enumerate(train_loader):
        image, label = data
        optimizer.zero_grad()
        
        output = baseline_model(image)
        __, predicts = torch.max(output, 1)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        
        total_image += label.size(0)
        correct_image += (predicts == label).sum().item()
        running_loss += loss.item()
        if i%100==0:
            print('batch:{}/{}, accuracy:{}'.format(i, len(train_loader), correct_image/total_image*100), end='\r')
    print('Epoch:{}, loss:{}, accuracy:{}'.format(
        epoch+1, running_loss/len(train_loader), correct_image/total_image*100
    ))
    loss_p=np.append(loss_p, running_loss/len(train_loader))
    accuracy_p=np.append(accuracy_p, correct_image/total_image*100)

    #validataion
    baseline_model.eval()
    validation_loss = 0
    validation_total_image = 0
    validation_correct_image = 0
    for i, data in enumerate(validation_loader):
        validation_image, validation_label = data
        with torch.no_grad():
            output = baseline_model(image)
            __, predict = torch.max(output, 1)
            loss = criterion(output, validation_label)
            
            validation_total_image += validation_label.size(0)
            validation_correct_image += (predict == validation_label).sum().item()
            validation_loss += loss

Epoch:1, loss:1.5133182077777385, accuracy:44.568000000000005
Epoch:2, loss:1.2607658039480447, accuracy:55.054
Epoch:3, loss:1.159048721614033, accuracy:59.019999999999996
